<a href="https://colab.research.google.com/github/gchit21/ML4/blob/main/ML_HW4_2LayerCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kaggle competition and wandb setup**


**Kaggle**

In [1]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge


 88% 250M/285M [00:00<00:00, 748MB/s] 
100% 285M/285M [00:02<00:00, 121MB/s]
Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


**wandb**

In [5]:
!pip install wandb onnx -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.9 MB/s eta 0:00:00


In [6]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gchit21 (gchit21-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# **Model**

In [7]:
import os
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
df = pd.read_csv("icml_face_data.csv")

**Turn the pixel string into an numpy Array and then into an Image**

In [9]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

from PIL import Image

import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

class FERDataset(Dataset):
    def __init__(self, csv_file, usage, transform=None):
        self.data = pd.read_csv(csv_file)
        self.data = self.data[self.data[' Usage'] == usage]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = np.fromstring(self.data.iloc[idx][' pixels'], sep=' ', dtype=np.uint8).reshape(48, 48)
        image = Image.fromarray(pixels)
        label = int(self.data.iloc[idx]['emotion'])

        if self.transform:
            image = self.transform(image)

        return image, label


In [10]:
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self,kernels,kernel_size,classes=7,drop_threshold=0.2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, kernels[0], kernel_size=kernel_size, padding=1)
        self.conv2 = nn.Conv2d(kernels[0], kernels[1], kernel_size=kernel_size, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(drop_threshold)
        self.fc1 = nn.Linear(kernels[1] * 12 * 12, 128)
        self.fc2 = nn.Linear(128, classes)  # 7 emotion classes


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 48 -> 24
        x = self.pool(F.relu(self.conv2(x)))  # 24 -> 12
        x = self.dropout(x)
        x = x.view(-1, 64 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [11]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        example_ct=0
        batch_ct=0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            loss, outputs = train_batch(images,labels,model,optimizer,criterion)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
        wandb.log({
          "epoch": num_epochs,
          "train/loss": round(running_loss / len(train_loader),4),
          "train/accuracy": round(100 * correct / total,2)
        })

def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss,outputs


In [12]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

In [13]:
def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    wandb.log({"Test Accuracy": round(100 * correct / total, 2)})

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [14]:
def make(config):

    transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),])

    # Make the data
    train =  FERDataset("icml_face_data.csv", usage="Training", transform=transform)
    val =  FERDataset("icml_face_data.csv", usage="PublicTest", transform=transforms.ToTensor())

    train_loader = DataLoader(train, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=config.batch_size, shuffle=False)

    # Make the model
    model = SimpleCNN(config.kernels,config.kernel_size,config.classes,config.drop_threshold).to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)

    return model, train_loader, val_loader, criterion, optimizer

In [15]:
def model_pipeline(hyperparameters, run_name):

    # tell wandb to get started
    with wandb.init(project="2-Layer", name=run_name, config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, val_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      train_model(model, train_loader, criterion, optimizer, device, config.epochs)

      # and test its final performance
      test_model(model, val_loader, device)

    return model

In [26]:
config = dict(
    epochs=10,
    classes=7,
    kernels=[32, 64],
    kernel_size = 3,
    batch_size=100,
    learning_rate=0.001,
    drop_threshold=0.35,
    dataset="icml_face_data",
    architecture="CNN")

name ="run_25"


model = model_pipeline(config,name)

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.35, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)
Loss after 02400 examples: 1.790
Loss after 04900 examples: 1.721
Loss after 07400 examples: 1.706
Loss after 09900 examples: 1.656
Loss after 12400 examples: 1.657
Loss after 14900 examples: 1.727
Loss after 17400 examples: 1.612
Loss after 19900 examples: 1.611
Loss after 22400 examples: 1.671
Loss after 24900 examples: 1.690
Loss after 27400 examples: 1.599
Epoch 1/10, Loss: 1.6984, Accuracy: 32.16%
Loss after 02400 examples: 1.615
Loss after 04900 examples: 1.450
Loss after 07400 examples: 1.520
Loss after 09900 examples: 1.701
Loss after 12400 examples: 1.480
Loss after 

wandb: WARNING Tried to log to step 2400 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 4900 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 7400 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 9900 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 12400 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be

Loss after 19900 examples: 1.520
Loss after 22400 examples: 1.520
Loss after 24900 examples: 1.539
Loss after 27400 examples: 1.622
Epoch 2/10, Loss: 1.5594, Accuracy: 39.56%
Loss after 02400 examples: 1.504
Loss after 04900 examples: 1.527
Loss after 07400 examples: 1.666


wandb: WARNING Tried to log to step 19900 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 22400 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 24900 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 27400 that is less than the current step 27401. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 2400 that is less than the current step 27402. Steps must be monotonically increasing, so this data will

Loss after 09900 examples: 1.551
Loss after 12400 examples: 1.478
Loss after 14900 examples: 1.525
Loss after 17400 examples: 1.349
Loss after 19900 examples: 1.459
Loss after 22400 examples: 1.540
Loss after 24900 examples: 1.602
Loss after 27400 examples: 1.507
Epoch 3/10, Loss: 1.4747, Accuracy: 43.50%
Loss after 02400 examples: 1.323


...

wandb: WARNING Tried to log to step 19900 that is less than the current step 27408. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 22400 that is less than the current step 27408. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 24900 that is less than the current step 27408. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 27400 that is less than the current step 27408. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 2400 that is less than the current step 27409. Steps must be monotonically increasing, so this data will

Loss after 09900 examples: 1.167
Loss after 12400 examples: 1.155
Loss after 14900 examples: 1.300
Loss after 17400 examples: 1.108
Loss after 19900 examples: 1.144
Loss after 22400 examples: 1.250
Loss after 24900 examples: 1.376
Loss after 27400 examples: 1.307
Epoch 10/10, Loss: 1.2232, Accuracy: 53.51%


wandb: WARNING Tried to log to step 9900 that is less than the current step 27409. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 12400 that is less than the current step 27409. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 14900 that is less than the current step 27409. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 17400 that is less than the current step 27409. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 19900 that is less than the current step 27409. Steps must be monotonically increasing, so this data will

Test Accuracy: 51.30%


Test Accuracy,▁
epoch,▁▁▁▁▁▁▁▁▁▁██████████
loss,█▅▅▃▃▆▁▁▄▄▁
train/accuracy,▁▃▅▅▆▆▇▇██
train/loss,█▆▅▄▃▃▂▂▁▁
Test Accuracy,51.3
epoch,10
loss,1.59899
train/accuracy,53.51
train/loss,1.2232
